# 💻 Installing necessary libraries
- `boto3` AWS operations
- `efficientnet` Neural network used for image classification:

In [ ]:
!pip install boto3
!pip install efficientnet

# 🔒 Retrieving AWS Credentials and configuring S3 connection

In [7]:
from google.colab import drive
import boto3
drive.mount('/content/drive')

CREDENTIALS_PATH = ''

def get_credential_value(lines: list, line_number: int):
    return lines[line_number].split('=')[1].strip()

with open(CREDENTIALS_PATH, 'r') as credentials_file:
    lines = credentials_file.readlines()

    aws_access_key_id = get_credential_value(lines, 0)
    aws_secret_access_key = get_credential_value(lines, 1)
    aws_region_name = get_credential_value(lines, 2)

s3 = boto3.resource(
    service_name='s3',
    region_name=aws_region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 🞜 Downloading the themes stored on S3 